In [ ]:
import requests

url = 'http://192.168.100.222:8888/infer'

wav_file_path = '/home/tuyendv/E2E-R/wav/classical.wav'

# Gửi request POST đến API
files = {'file': open(wav_file_path, 'rb')}
data = {'transcript': 'classical'}
response = requests.post(url, files=files, data=data)
